## Import Speech Processing Libraries
- `Librosa` for Audio Processing
- `torch` for Pytorch
- `CTC` stands for Connectionist Temporal Classification
    - Self-supervised method to map input sound sequences to an ouput (embedding) when alignment is not clear or available
- Wav2Vec2Processor preprocess audio for HuBERT model

In [1]:
# Core Libraries
import librosa
import torch
from transformers import HubertForCTC, Wav2Vec2Processor

# Utilitiies
import numpy as np
import matplotlib.pyplot as plt

/Users/kyleng/B_Organized/A_School/Ling_487/clean_code/HuBERT-Analysis/.transformers-venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/kyleng/B_Organized/A_School/Ling_487/clean_code/HuBERT-Analysis/.transformers-venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Wav2Vec2Processor
- `960h` refers to 960 hours of Librispeech training
- Librispeech is an ASR corpus sampled at 60kHz

In [2]:
# Import HuBERT Model
processor = Wav2Vec2Processor.from_pretrained(
    "facebook/wav2vec2-base-960h"
)

In [3]:
model = HubertForCTC.from_pretrained(
    "facebook/hubert-large-ls960-ft", 
    output_attentions=True, 
    output_hidden_states=True
)

Some weights of the model checkpoint at facebook/hubert-large-ls960-ft were not used when initializing HubertForCTC: ['hubert.encoder.pos_conv_embed.conv.weight_g', 'hubert.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing HubertForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing HubertForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of HubertForCTC were not initialized from the model checkpoint at facebook/hubert-large-ls960-ft and are newly initialized: ['hubert.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'hubert.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably TRAIN this model on a down-